In [22]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [23]:
spark = SparkSession.builder.appName('Insights').getOrCreate()

In [32]:
df = spark.read.parquet("../../data/processed/cleanedData.parquet")

In [39]:
dfGrouped = df.groupBy("Customer_ID").agg(
    F.array_sort(F.collect_list("Date")).alias("Purchase_Dates"),
    F.round(F.sum("Total_Amount") ,2).alias("Total_Spend"),
    F.sum("Total_Purchases").alias("Total_Purchases")
)
dfGrouped = dfGrouped.withColumn("Last_Purchase", F.element_at(F.col("Purchase_Dates"), -1)) \
                     .withColumn("Second_Last_Purchase", F.when(F.size(F.col("Purchase_Dates")) > 1, 
                                                                F.element_at(F.col("Purchase_Dates"), -2)))

dfGrouped = dfGrouped.withColumn("Recency", 
                                 F.when(F.col("Second_Last_Purchase").isNull(), 0)
                                  .otherwise(F.datediff(F.col("Last_Purchase"), F.col("Second_Last_Purchase"))))
dfGrouped = dfGrouped.drop('Purchase_Dates')
dfGrouped = dfGrouped.where(F.col('Recency')>0)
latestDate = df.select(F.max('Date')).collect()[0][0]
churnDf = dfGrouped.withColumn(
    "Churn", 
    F.when(F.datediff(F.lit(latestDate), F.col("Last_Purchase")) > 90, 1).otherwise(0)
)
churnDf = churnDf.withColumn('Avg_Order_Value', F.round(F.col('Total_Spend') / F.col('Total_Purchases'), 2))
churnDf = churnDf.drop('Last_Purchase', 'Second_Last_Purchase')
churnDf.show()


+-----------+-----------+---------------+-------+-----+---------------+
|Customer_ID|Total_Spend|Total_Purchases|Recency|Churn|Avg_Order_Value|
+-----------+-----------+---------------+-------+-----+---------------+
|      10009|    1997.33|             13|     38|    1|         153.64|
|      10038|    5194.43|             22|     93|    1|         236.11|
|      10039|     7291.8|             30|     26|    1|         243.06|
|      10052|    5157.83|             18|     40|    1|         286.55|
|      10058|    9831.52|             33|     79|    0|         297.92|
|      10077|     5289.4|             13|     37|    1|         406.88|
|      10088|     8172.4|             22|     60|    0|         371.47|
|      10096|    3397.38|             12|   1886|    1|         283.12|
|      10104|     1742.4|             20|     37|    0|          87.12|
|      10110|     642.28|              4|   3793|    1|         160.57|
|      10128|    8834.12|             30|      1|    1|         

In [16]:
#Customer is Churned if no purchase in last 3 months
customerLastPurchase = df.groupBy('Customer_ID').agg(F.max(F.col('Date')).alias('Last_Purchase'), F.sum(F.col('Total_Amount')).alias('Total_Spend'), F.sum(F.col('Total_Purchases')).alias('Total_Purchases')).orderBy('Last_Purchase')
latestDate = customerLastPurchase.select(F.max('Last_Purchase')).collect()[0][0]
print(latestDate)
churnDf = customerLastPurchase.withColumn(
    "Churn", 
    F.when(F.datediff(F.lit(latestDate), F.col("Last_Purchase")) > 90, 1).otherwise(0)
)
churnDf.show()

2024-02-29
+-----------+-------------+------------------+---------------+-----+
|Customer_ID|Last_Purchase|       Total_Spend|Total_Purchases|Churn|
+-----------+-------------+------------------+---------------+-----+
|      39352|   2001-01-24| 1880.511474609375|              7|    1|
|      80322|   2001-01-24|1512.3026123046875|              6|    1|
|      87823|   2001-01-24| 2405.681884765625|              9|    1|
|      37647|   2001-01-24|    2394.283203125|              7|    1|
|      12437|   2001-01-24| 683.1224365234375|              2|    1|
|      42443|   2001-01-24|45.104061126708984|              3|    1|
|      79528|   2001-01-24| 89.44783020019531|              1|    1|
|      79613|   2001-01-24| 808.9075927734375|              3|    1|
|      31899|   2001-01-24|     3433.47265625|             10|    1|
|      38168|   2001-01-24| 2099.823486328125|              5|    1|
|      42888|   2001-01-24|   1847.8388671875|              6|    1|
|      62773|   2001-01